In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "C:\\Users\\i076453\\Downloads\\tech\\ml\\cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """ read data from data file """
    with open(filename, 'rb') as f:
        data = cPickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)     # 不做filter, 将所有的数据都加载
            all_labels.append(labels)
        self._data = np.vstack(all_data)  # 纵向合并成
        self._data = self._data / 127.5 - 1 # 进行归一化 
            
        self._labels = np.hstack(all_labels) # 横向合并
        
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
        
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)] 
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
x = tf.placeholder(tf.float32, [None, 3072]) # data 的placeholder, None表示不确定的

# y的shape是 [0.6,5,3]
y = tf.placeholder(tf.int64, [None]) # labels的placeholder
# (3072, 10)
w = tf.get_variable('w', [x.get_shape()[-1], 10],
                    initializer=tf.random_normal_initializer(0,1))   # 这里从1修改为10
# (10, )
b = tf.get_variable('b', [10], 
                   initializer=tf.constant_initializer(0.0))
# [None, 3071] * [3072, 10] = [None, 10]
y_ = tf.matmul(x, w) + b

# mean square loss 平方差交叉函数
"""
# course: 1+ e^x
# api: e^x / sum(e^x)
# [[0.01, 0.09,... 0.03], []]
p_y = tf.nn.softmax(y_)
# 5 -> [0,0,0,0,0,1,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
# 还是用平方差来求解，而不是交叉熵
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""
# 交叉熵
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 它做了下面三件事
# y_ -> softmax
# y -> one hot
# loss = ylogy_

# bool 
predict = tf.argmax(y_, 1) # 在第二个维度求最大值

# [1,0,0,1,0,0,1,1...]
correct_predict = tf.equal(predict, y)

# 准确率
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float64))

# 有了loss目标函数和准确率后，再去做一个
# AdamOptimizer 是一个剔除向量的变种，1e-3就是10的-3次方0.001 
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size) 
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 500, loss: 20.99282, acc: 0.15000
[Train] Step: 1000, loss: 17.96407, acc: 0.25000
[Train] Step: 1500, loss: 9.75291, acc: 0.25000
[Train] Step: 2000, loss: 13.38364, acc: 0.15000
[Train] Step: 2500, loss: 10.37234, acc: 0.20000
[Train] Step: 3000, loss: 12.88054, acc: 0.20000
[Train] Step: 3500, loss: 5.72834, acc: 0.30000
[Train] Step: 4000, loss: 8.51628, acc: 0.35000
[Train] Step: 4500, loss: 7.09195, acc: 0.25000
[Train] Step: 5000, loss: 7.53508, acc: 0.35000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.25900
[Train] Step: 5500, loss: 7.38133, acc: 0.25000
[Train] Step: 6000, loss: 6.52366, acc: 0.20000
[Train] Step: 6500, loss: 6.95883, acc: 0.25000
[Train] Step: 7000, loss: 4.96340, acc: 0.30000
[Train] Step: 7500, loss: 6.01076, acc: 0.30000
[Train] Step: 8000, loss: 4.79835, acc: 0.30000
[Train] Step: 8500, loss: 6.36990, acc: 0.20000
[Train] Step: 9000, loss: 3.58377, acc: 0.45000
[Train] Step: 9500, loss: 3.86228, acc: 0.40000
[Train] Step: 10000, loss: 7.